In [ ]:
import os
import datetime
import time

import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.preprocessing.sequence import TimeseriesGenerator


mpl.rcParams['figure.figsize'] = (10, 8)
mpl.rcParams['axes.grid'] = False


In [ ]:

data_2 = pd.read_csv('./CSI_Preprocessing/output/zero4_outputs.csv')
data_2 = data_2.drop([data_2.columns[0]], axis='columns')
data_2 = data_2.iloc[:, :-3]
data_2.reset_index(drop=True, inplace=True)
# unit='s' to convert it into epoch time
data_2['Time'] = pd.to_datetime(data_2['Time'])

date_time = data_2['Time'].dt.strftime('%S')


# checking our dataframe once again
data_2.head()


In [ ]:
plot_cols = ['Subcar_6', 'Subcar_7', 'Subcar_9',
             'Subcar_10', 'Subcar_11', 'Subcar_13']
plot_features = data_2[plot_cols]
plot_features.index = date_time
_ = plot_features.plot(subplots=False)


In [ ]:
plot_cols = ['Subcar_6', 'Subcar_7', 'Subcar_8']
plot_features = data_2[plot_cols]
plot_features.index = date_time
_ = plot_features.plot(subplots=True)

plot_features = data_2[plot_cols][:480]
plot_features.index = date_time[:480]
_ = plot_features.plot(subplots=True)


In [ ]:
# Set the date column as the index of your DataFrame meat
data_2 = data_2.set_index('Time')

# Print the summary statistics of the DataFrame
print(data_2.describe())


In [ ]:
df_input = data_2[['Subcar_6', 'Subcar_7', 'Subcar_9',
                   'Subcar_10', 'Subcar_11', 'Subcar_13']]
df_input.head()


In [ ]:
df_input.describe()


In [ ]:
df_input.query("Subcar_7 > 511")


In [ ]:
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(df_input)
data_scaled


In [ ]:
features = data_scaled
target = data_scaled[:, 1]


In [ ]:
TimeseriesGenerator(features, target, length=6,
                    sampling_rate=1, batch_size=1)[1]


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(
    features, target, test_size=0.20, random_state=120, shuffle=False)
print('X_train.shape: ', x_train.shape)
print('X_test.shape: ', x_test.shape)


In [ ]:
win_length = 144  # 1 day = 144 of data, 5 day = 720 of data
batch_size = 32
num_features = 6
train_generator = tf.keras.preprocessing.sequence.TimeseriesGenerator(
    x_train, y_train, length=win_length, sampling_rate=1, batch_size=batch_size)
test_generator = tf.keras.preprocessing.sequence.TimeseriesGenerator(
    x_test, y_test, length=win_length, sampling_rate=1, batch_size=batch_size)

In [ ]:
train_generator[0]


In [ ]:
lstm_model = tf.keras.Sequential([
    tf.keras.layers.LSTM(128, input_shape=(
        win_length, num_features), return_sequences=True),
    tf.keras.layers.LeakyReLU(alpha=0.5),
    tf.keras.layers.LSTM(128, return_sequences=True),
    tf.keras.layers.LeakyReLU(alpha=0.5),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.LSTM(64, return_sequences=False),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(1)
])

lstm_model.summary()


In [ ]:
cp1 = ModelCheckpoint('../save/lstm_model/', save_best_only=True)
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=2,
    mode='min'
)
lstm_model.compile(loss=tf.keras.losses.MeanSquaredError(),
                   optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
                   metrics=[tf.keras.metrics.MeanAbsoluteError()])


In [ ]:
history = lstm_model.fit_generator(
    train_generator, epochs=50,
    validation_data=test_generator,
    shuffle=False,
    callbacks=[early_stopping, cp1 ])


In [ ]:
lstm_eval = lstm_model.evaluate_generator(
    test_generator, verbose=0
)


In [ ]:
predictions_lstm = lstm_model.predict_generator(test_generator)


In [ ]:
from tensorflow.keras.models import load_model
gru_model = load_model('../save/lstm_model/')

In [ ]:
val_predictions = lstm_model.predict(x_test).flatten()
val_results = pd.DataFrame(data={'Val predictions': val_predictions, 'Actuals': y_test})
val_results

In [ ]:
plt.plot(train_results['Train Predictions'])
plt.plot(train_results['Actuals'])

In [ ]:
predictions_lstm.shape[0]


In [ ]:
predictions_lstm


In [ ]:
y_test


In [ ]:
x_test


In [ ]:
x_test[:, 1:][win_length:]


In [ ]:
df_pred_lstm = pd.concat([pd.DataFrame(predictions_lstm),
                    pd.DataFrame(x_test[:, 1:][win_length:])], axis=1)
df_pred_lstm


In [ ]:
# To the original format
rev_trans = scaler.inverse_transform(df_pred_lstm)
rev_trans


In [ ]:
df_final_lstm = df_input[predictions_lstm.shape[0]*-1:]


In [ ]:
df_final_lstm.count()


In [ ]:
df_final_lstm['Subcar_7_pred'] = rev_trans[:, 1]
df_final_lstm


In [ ]:
plot_cols = [ 'Subcar_7','Subcar_7', 'Subcar_9',
             'Subcar_10', 'Subcar_11', 'Subcar_13']
plot_features = df_input[plot_cols]
plot_features.index = date_time
_ = plot_features.plot(subplots=False)

df_final_lstm[['Subcar_7', 'Subcar_7_pred']]
df_final_lstm.index = date_time[:617]
_ = df_final_lstm.plot(subplots=False)


In [ ]:
gru_model = tf.keras.Sequential([
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(
        128, input_shape=(win_length, num_features), return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])


In [ ]:
cp3 = ModelCheckpoint('../save/gru_model/', save_best_only=True)
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=2,
    mode='min'
)
gru_model.compile(loss=tf.keras.losses.MeanSquaredError(),
                  optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
                  metrics=[tf.keras.metrics.MeanAbsoluteError()])


In [ ]:
history_2 = gru_model.fit_generator(
    train_generator, epochs=50,
    validation_data=test_generator,
    shuffle=False,
    callbacks=[early_stopping
               , cp3])


In [ ]:
from tensorflow.keras.models import load_model
gru_model = load_model('../save/gru_model/')

In [ ]:
train_predictions = gru_model.predict(x_train).flatten()
train_results = pd.DataFrame(data={'Train Predictions': train_predictions, 'Actuals': y_train})
train_results

In [ ]:
plt.plot(train_results['Train Predictions'])
plt.plot(train_results['Actuals'])

In [ ]:
val_predictions = gru_model.predict(x_test).flatten()
val_results = pd.DataFrame(data={'Val predictions': val_predictions, 'Actuals': y_test})
val_results

In [ ]:
plt.plot(train_results['Train Predictions'])
plt.plot(train_results['Actuals'])

In [ ]:
gru_eval = gru_model.evaluate_generator(
    test_generator, verbose=0
)


In [ ]:
predictions_gru = gru_model.predict_generator(test_generator)


In [ ]:
predictions_gru.shape[0]


In [ ]:
predictions_gru


In [ ]:
df_pred_gru = pd.concat([pd.DataFrame(predictions_gru),
                    pd.DataFrame(x_test[:, 1:][win_length:])], axis=1)
df_pred_gru


In [ ]:
# To the original format
rev_trans = scaler.inverse_transform(df_pred_gru)
rev_trans


In [ ]:
df_final_gru = df_input[predictions_gru.shape[0]*-1:]


In [ ]:
df_final_gru['Subcar_7_pred'] = rev_trans[:, 1]
df_final_gru


In [ ]:

df_final_gru[['Subcar_7', 'Subcar_7_pred']]
df_final_gru.index = date_time[:617]
_ = df_final_gru.plot(subplots=False)

df_final_lstm[['Subcar_7','Subcar_7_pred']]
df_final_gru.index = date_time[:617]
_ = df_final_gru.plot(subplots=False)




In [ ]:
train_predictions = model3.predict(x_train).flatten()
train_results = pd.DataFrame(data={'Train Predictions': train_predictions, 'Actuals': y_train})
train_results

In [ ]:
plt.plot(train_results['Train Predictions'])
plt.plot(train_results['Actuals'])

In [ ]:
val_predictions = model3.predict(x_val).flatten()
val_results = pd.DataFrame(data={'Val predictions': val_predictions, 'Actuals': y_val})
val_results

In [ ]:
plt.plot(val_results['Val predictions'])
plt.plot(val_results['Actuals'])